In [25]:
from sklearn import preprocessing
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import shutil
import os
import requests
import base64


# Encode text values to dummy variables(i.e. [1,0,0],[0,1,0],[0,0,1] for red,green,blue)
def encode_text_dummy(df, name):
    dummies = pd.get_dummies(df[name])
    for x in dummies.columns:
        dummy_name = "{}-{}".format(name, x)
        df[dummy_name] = dummies[x]
    df.drop(name, axis=1, inplace=True)


# Encode text values to a single dummy variable.  The new columns (which do not replace the old) will have a 1
# at every location where the original column (name) matches each of the target_values.  One column is added for
# each target value.
def encode_text_single_dummy(df, name, target_values):
    for tv in target_values:
        l = list(df[name].astype(str))
        l = [1 if str(x) == str(tv) else 0 for x in l]
        name2 = "{}-{}".format(name, tv)
        df[name2] = l


# Encode text values to indexes(i.e. [1],[2],[3] for red,green,blue).
def encode_text_index(df, name):
    le = preprocessing.LabelEncoder()
    df[name] = le.fit_transform(df[name])
    return le.classes_


# Encode a numeric column as zscores
def encode_numeric_zscore(df, name, mean=None, sd=None):
    if mean is None:
        mean = df[name].mean()

    if sd is None:
        sd = df[name].std()

    df[name] = (df[name] - mean) / sd


# Convert all missing values in the specified column to the median
def missing_median(df, name):
    med = df[name].median()
    df[name] = df[name].fillna(med)


# Convert all missing values in the specified column to the default
def missing_default(df, name, default_value):
    df[name] = df[name].fillna(default_value)


# Convert a Pandas dataframe to the x,y inputs that TensorFlow needs
def to_xy(df, target):
    result = []
    for x in df.columns:
        if x != target:
            result.append(x)
    # find out the type of the target column.  Is it really this hard? :(
    target_type = df[target].dtypes
    target_type = target_type[0] if hasattr(target_type, '__iter__') else target_type
    # Encode to int for classification, float otherwise. TensorFlow likes 32 bits.
    if target_type in (np.int64, np.int32):
        # Classification
        dummies = pd.get_dummies(df[target])
        return df.as_matrix(result).astype(np.float32), dummies.as_matrix().astype(np.float32)
    else:
        # Regression
        return df.as_matrix(result).astype(np.float32), df.as_matrix([target]).astype(np.float32)

# Nicely formatted time string
def hms_string(sec_elapsed):
    h = int(sec_elapsed / (60 * 60))
    m = int((sec_elapsed % (60 * 60)) / 60)
    s = sec_elapsed % 60
    return "{}:{:>02}:{:>05.2f}".format(h, m, s)


# Regression chart.
def chart_regression(pred,y,sort=True):
    t = pd.DataFrame({'pred' : pred, 'y' : y.flatten()})
    if sort:
        t.sort_values(by=['y'],inplace=True)
    a = plt.plot(t['y'].tolist(),label='expected')
    b = plt.plot(t['pred'].tolist(),label='prediction')
    plt.ylabel('output')
    plt.legend()
    plt.show()

# Remove all rows where the specified column is +/- sd standard deviations
def remove_outliers(df, name, sd):
    drop_rows = df.index[(np.abs(df[name] - df[name].mean()) >= (sd * df[name].std()))]
    df.drop(drop_rows, axis=0, inplace=True)


# Encode a column to a range between normalized_low and normalized_high.
def encode_numeric_range(df, name, normalized_low=-1, normalized_high=1,
                         data_low=None, data_high=None):
    if data_low is None:
        data_low = min(df[name])
        data_high = max(df[name])

    df[name] = ((df[name] - data_low) / (data_high - data_low)) \
               * (normalized_high - normalized_low) + normalized_low
        
# This function submits an assignment.  You can submit an assignment as much as you like, only the final
# submission counts.  The paramaters are as follows:
# data - Pandas dataframe output.
# key - Your student key that was emailed to you.
# no - The assignment class number, should be 1 through 1.
# source_file - The full path to your Python or IPYNB file.  This must have "_class1" as part of its name.  
# .             The number must match your assignment number.  For example "_class2" for class assignment #2.
def submit(data,key,no,source_file=None):
    if source_file is None and '__file__' not in globals(): raise Exception('Must specify a filename when a Jupyter notebook.')
    if source_file is None: source_file = __file__
    suffix = '_class{}'.format(no)
    if suffix not in source_file: raise Exception('{} must be part of the filename.'.format(suffix))
    with open(source_file, "rb") as image_file:
        encoded_python = base64.b64encode(image_file.read()).decode('ascii')
    ext = os.path.splitext(source_file)[-1].lower()
    if ext not in ['.ipynb','.py']: raise Exception("Source file is {} must be .py or .ipynb".format(ext))
    r = requests.post("https://api.heatonresearch.com/assignment-submit",
        headers={'x-api-key':key}, json={'csv':base64.b64encode(data.to_csv(index=False).encode('ascii')).decode("ascii"),
        'assignment': no, 'ext':ext, 'py':encoded_python})
    if r.status_code == 200:
        print("Success: {}".format(r.text))
    else: print("Failure: {}".format(r.text))

In [26]:
import os
import re
import sklearn

import pandas as pd
import numpy as np
import tensorflow as tf

from sklearn import metrics
from sklearn.linear_model import LassoCV
from sklearn.model_selection import train_test_split, KFold
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import regularizers

path = './data'

filename_test = os.path.join(path,"test.csv")
filename_train = os.path.join(path,"train.csv")
filename_sample = os.path.join(path,"sample.csv")
filename_submit = os.path.join(path,"submit.csv")
filename_checkpoint = os.path.join(path,"checkpoint.hdf5")

df_train = pd.read_csv(filename_train,na_values=['NA','?'])

np.random.seed(42) # Uncomment this line to get the same shuffle each time
df_train = df_train.reindex(np.random.permutation(df_train.index))
df_train.reset_index(inplace=True, drop=True)

In [27]:
# Encode Features

# Remove all rows where the specified column is +/- sd standard deviations
def extract_and_encode_features(df):
    color_regex='(?P<color>red|blue|green|yellow|orange|pink|black|brown|white)'
    df['color'] = df.name.str.extract(color_regex, flags=re.IGNORECASE, expand=False)

    quality_regex='(?P<quality>generic|medium\shigh\squality|high\squality)'
    df['quality'] = df.name.str.extract(quality_regex, flags=re.IGNORECASE, expand=False)

    size_regex='(?P<size>tiny|small|medium|large)'
    df['size'] = df.name.str.extract(size_regex, flags=re.IGNORECASE, expand=False)

    item_regex='(?P<item>paperclips|paperweights|ink\spens|pencils|stapler|tablets|thumbtacks|post\sit\snotes)'
    df['item'] = df.name.str.extract(item_regex, flags=re.IGNORECASE, expand=False)
    
    for column in ['pack','weight','height','width','length']:
        missing_median(df,column)
    

extract_and_encode_features(df_train)
ids_train = df_train['id']
df_train.drop('id',1,inplace=True)

names_train = df_train['name']
df_train.drop('name',1,inplace=True)

In [28]:
missing_default(df_train, "quality", "Unknown")
missing_default(df_train, "size", "Unknown")
missing_default(df_train, "color", "Unknown")

In [29]:
# encode text/categorical features
for column in ['manufacturer','color','quality','size','item']:
    encode_text_dummy(df_train,column)

In [30]:
x,y = to_xy(df_train,'cost')
# Cross-Validate
#kf = KFold(5)

# Used before KFold
x_train, x_test, y_train, y_test = train_test_split(    
    x, y, test_size=0.25, random_state=45)

#oos_y = []
#oos_pred = []
#fold = 0
checkpoint = ModelCheckpoint(filepath=filename_checkpoint, verbose=0, save_best_only=True)
#for train, test in kf.split(x):
#    fold+=1
#    print("Fold #{}".format(fold))
        
#x_train = x[train]
#y_train = y[train]
#x_test = x[test]
#y_test = y[test]
    
model = Sequential()
model.add(Dense(50, input_dim=x.shape[1], activation='relu')) # Hidden 1
model.add(Dropout(0.01)) # Dropout Layer
model.add(Dense(25, activation='relu')) # Hidden 2
model.add(Dense(10, 
            kernel_regularizer=regularizers.l2(0.01),
            activity_regularizer=regularizers.l1(0.01),activation='relu')) # Hidden 3 w/regularization
model.add(Dense(1)) # Output
model.compile(loss='mean_squared_error', optimizer='adam')
monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=100, verbose=1, mode='auto')
model.fit(x_train,y_train,validation_data=(x_test,y_test),callbacks=[monitor,checkpoint],verbose=1,epochs=1000)
    
pred = model.predict(x_test)
    
    #oos_y.append(y_test)
    #oos_pred.append(pred)        

    # Measure this fold's RMSE
score = np.sqrt(metrics.mean_squared_error(pred,y_test))
print("Fold score (RMSE): {}".format(score))

Train on 7500 samples, validate on 2500 samples
Epoch 1/1000
7500/7500 [==============================] - 2s 325us/step - loss: 9155.4236 - val_loss: 5782.8448
Epoch 2/1000
7500/7500 [==============================] - 1s 132us/step - loss: 5176.3386 - val_loss: 4501.6916
Epoch 3/1000
7500/7500 [==============================] - 1s 131us/step - loss: 4862.0770 - val_loss: 4204.9721
Epoch 4/1000
7500/7500 [==============================] - 1s 118us/step - loss: 4525.5233 - val_loss: 4144.0036
Epoch 5/1000
7500/7500 [==============================] - 1s 115us/step - loss: 4321.6431 - val_loss: 4018.3748
Epoch 6/1000
7500/7500 [==============================] - 1s 109us/step - loss: 4229.9313 - val_loss: 4363.3942
Epoch 7/1000
7500/7500 [==============================] - 1s 122us/step - loss: 4108.7489 - val_loss: 3689.0416
Epoch 8/1000
7500/7500 [==============================] - 1s 120us/step - loss: 4093.8548 - val_loss: 4050.7764
Epoch 9/1000
7500/7500 [==============================] 

7500/7500 [==============================] - 1s 109us/step - loss: 387.1007 - val_loss: 285.9661
Epoch 75/1000
7500/7500 [==============================] - 1s 135us/step - loss: 411.9960 - val_loss: 253.2748
Epoch 76/1000
7500/7500 [==============================] - 1s 106us/step - loss: 440.9293 - val_loss: 293.2301
Epoch 77/1000
7500/7500 [==============================] - 1s 107us/step - loss: 332.8778 - val_loss: 283.3978
Epoch 78/1000
7500/7500 [==============================] - 1s 128us/step - loss: 357.2858 - val_loss: 379.7809
Epoch 79/1000
7500/7500 [==============================] - 1s 119us/step - loss: 471.3620 - val_loss: 228.6587
Epoch 80/1000
7500/7500 [==============================] - 1s 128us/step - loss: 335.8453 - val_loss: 234.1500
Epoch 81/1000
7500/7500 [==============================] - 1s 128us/step - loss: 398.8461 - val_loss: 244.1026
Epoch 82/1000
7500/7500 [==============================] - 1s 147us/step - loss: 521.0303 - val_loss: 238.7647
Epoch 83/1000
7

7500/7500 [==============================] - 1s 89us/step - loss: 259.9567 - val_loss: 199.2895
Epoch 148/1000
7500/7500 [==============================] - 1s 86us/step - loss: 267.6880 - val_loss: 1324.6417
Epoch 149/1000
7500/7500 [==============================] - 1s 89us/step - loss: 325.0933 - val_loss: 246.1542
Epoch 150/1000
7500/7500 [==============================] - 1s 89us/step - loss: 235.2346 - val_loss: 190.9162
Epoch 151/1000
7500/7500 [==============================] - 1s 86us/step - loss: 278.1364 - val_loss: 191.5142
Epoch 152/1000
7500/7500 [==============================] - 1s 89us/step - loss: 291.8154 - val_loss: 248.8168
Epoch 153/1000
7500/7500 [==============================] - 1s 91us/step - loss: 328.5075 - val_loss: 228.9461
Epoch 154/1000
7500/7500 [==============================] - 1s 88us/step - loss: 297.2035 - val_loss: 194.7753
Epoch 155/1000
7500/7500 [==============================] - 1s 90us/step - loss: 311.9067 - val_loss: 394.4882
Epoch 156/1000


7500/7500 [==============================] - 1s 91us/step - loss: 254.0571 - val_loss: 176.3701
Epoch 221/1000
7500/7500 [==============================] - 1s 91us/step - loss: 243.2415 - val_loss: 306.2147
Epoch 222/1000
7500/7500 [==============================] - 1s 90us/step - loss: 298.6537 - val_loss: 179.4571
Epoch 223/1000
7500/7500 [==============================] - 1s 89us/step - loss: 256.5423 - val_loss: 174.6243
Epoch 224/1000
7500/7500 [==============================] - 1s 93us/step - loss: 292.8609 - val_loss: 571.0708
Epoch 225/1000
7500/7500 [==============================] - 1s 96us/step - loss: 262.1494 - val_loss: 961.6266
Epoch 226/1000
7500/7500 [==============================] - 1s 96us/step - loss: 325.4087 - val_loss: 219.8183
Epoch 227/1000
7500/7500 [==============================] - 1s 93us/step - loss: 264.3772 - val_loss: 172.1804
Epoch 228/1000
7500/7500 [==============================] - 1s 88us/step - loss: 265.4892 - val_loss: 296.5762
Epoch 229/1000
7

Epoch 294/1000
7500/7500 [==============================] - 1s 89us/step - loss: 199.4011 - val_loss: 196.5704
Epoch 295/1000
7500/7500 [==============================] - 1s 93us/step - loss: 234.3886 - val_loss: 157.4095
Epoch 296/1000
7500/7500 [==============================] - 1s 89us/step - loss: 204.1511 - val_loss: 170.7450
Epoch 297/1000
7500/7500 [==============================] - 1s 89us/step - loss: 219.2959 - val_loss: 174.1794
Epoch 298/1000
7500/7500 [==============================] - 1s 92us/step - loss: 225.5935 - val_loss: 243.6278
Epoch 299/1000
7500/7500 [==============================] - 1s 95us/step - loss: 254.4209 - val_loss: 154.9710
Epoch 300/1000
7500/7500 [==============================] - 1s 91us/step - loss: 224.5884 - val_loss: 193.9904
Epoch 301/1000
7500/7500 [==============================] - 1s 89us/step - loss: 200.7299 - val_loss: 172.4750
Epoch 302/1000
7500/7500 [==============================] - 1s 91us/step - loss: 195.0187 - val_loss: 218.9593
E

7500/7500 [==============================] - 1s 88us/step - loss: 208.0823 - val_loss: 163.2374
Epoch 368/1000
7500/7500 [==============================] - 1s 89us/step - loss: 199.8447 - val_loss: 269.3113
Epoch 369/1000
7500/7500 [==============================] - 1s 87us/step - loss: 217.3123 - val_loss: 176.7369
Epoch 370/1000
7500/7500 [==============================] - 1s 91us/step - loss: 253.2562 - val_loss: 310.9833
Epoch 371/1000
7500/7500 [==============================] - 1s 87us/step - loss: 219.5500 - val_loss: 741.6795
Epoch 372/1000
7500/7500 [==============================] - 1s 88us/step - loss: 231.3473 - val_loss: 162.7380
Epoch 373/1000
7500/7500 [==============================] - 1s 87us/step - loss: 251.8387 - val_loss: 170.9251
Epoch 374/1000
7500/7500 [==============================] - 1s 88us/step - loss: 228.8157 - val_loss: 231.1690
Epoch 375/1000
7500/7500 [==============================] - 1s 88us/step - loss: 228.7212 - val_loss: 240.1002
Epoch 376/1000
7

Epoch 441/1000
7500/7500 [==============================] - 1s 86us/step - loss: 178.1628 - val_loss: 172.1086
Epoch 442/1000
7500/7500 [==============================] - 1s 89us/step - loss: 181.3061 - val_loss: 176.8076
Epoch 443/1000
7500/7500 [==============================] - 1s 90us/step - loss: 180.1230 - val_loss: 154.2807
Epoch 444/1000
7500/7500 [==============================] - 1s 89us/step - loss: 254.3592 - val_loss: 156.6742
Epoch 445/1000
7500/7500 [==============================] - 1s 89us/step - loss: 205.1171 - val_loss: 167.1074
Epoch 446/1000
7500/7500 [==============================] - 1s 89us/step - loss: 250.9008 - val_loss: 195.4211
Epoch 447/1000
7500/7500 [==============================] - 1s 92us/step - loss: 187.8398 - val_loss: 149.8264
Epoch 448/1000
7500/7500 [==============================] - 1s 91us/step - loss: 187.5133 - val_loss: 143.5496
Epoch 449/1000
7500/7500 [==============================] - 1s 89us/step - loss: 174.9703 - val_loss: 186.7225
E

Epoch 515/1000
7500/7500 [==============================] - 1s 87us/step - loss: 493.0492 - val_loss: 158.6672
Epoch 516/1000
7500/7500 [==============================] - 1s 89us/step - loss: 219.7314 - val_loss: 619.5846
Epoch 517/1000
7500/7500 [==============================] - 1s 89us/step - loss: 265.2588 - val_loss: 164.6238
Epoch 518/1000
7500/7500 [==============================] - 1s 87us/step - loss: 301.2148 - val_loss: 1113.9368
Epoch 519/1000
7500/7500 [==============================] - 1s 89us/step - loss: 218.5282 - val_loss: 147.9368
Epoch 520/1000
7500/7500 [==============================] - 1s 92us/step - loss: 204.3601 - val_loss: 172.0854
Epoch 521/1000
7500/7500 [==============================] - 1s 90us/step - loss: 175.1660 - val_loss: 157.6186
Epoch 522/1000
7500/7500 [==============================] - 1s 89us/step - loss: 261.7439 - val_loss: 186.8081
Epoch 523/1000
7500/7500 [==============================] - 1s 91us/step - loss: 182.0323 - val_loss: 155.6250


In [31]:
model.load_weights(filename_checkpoint)
pred = model.predict(x_test)

# Measure RMSE error.  RMSE is common for regression.
score = np.sqrt(metrics.mean_squared_error(pred,y_test))
print("Final score (RMSE): {}".format(score))

#chart_regression(pred.flatten(),y_test)
#chart_regression(pred.flatten(),y_test,sort=False)

Final score (RMSE): 10.958354949951172


In [33]:
df_test = pd.read_csv(filename_test,na_values=['NA','?'])

extract_and_encode_features(df_test)
missing_default(df_test, "quality", "Unknown")
missing_default(df_test, "size", "Unknown")
missing_default(df_test, "color", "Unknown")

for column in ['manufacturer','color','quality','size','item']:
    encode_text_dummy(df_test,column)

ids_test = df_test['id']
df_test.drop('id',1,inplace=True)

names_test = df_test['name']
df_test.drop('name',1,inplace=True)

x_submit = df_test.as_matrix().astype(np.float32)
pred_submit = model.predict(x_submit)

In [34]:
df_submit = pd.DataFrame(pred_submit)
df_submit.insert(0,'id',ids_test)
df_submit.columns = ['id','cost']

In [35]:
df_submit.to_csv('submit_text_dummy_kagglecode5.csv', index=False)